In [50]:
from __future__ import print_function
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import pandas as pd
import numpy as np

import category_encoders as ce
import numpy as np
import pandas as pd
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import seaborn as sns
pd.set_option('display.max_columns', None)

In [65]:
encoder = ce.OneHotEncoder()

df = pd.read_csv("students-all.csv").iloc[:,1:]
y = df.romantic
df = df.drop("romantic", axis =1)

encoder.fit(df)

OneHotEncoder(cols=['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob',
                    'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid',
                    'activities', 'nursery', 'higher', 'internet', 'major'],
              drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=False,
              verbose=0)

In [66]:
df_one_hot = encoder.transform(df)

In [67]:
df_one_hot.columns

Index(['school_1', 'school_2', 'sex_1', 'sex_2', 'age', 'address_1',
       'address_2', 'famsize_1', 'famsize_2', 'Pstatus_1', 'Pstatus_2', 'Medu',
       'Fedu', 'Mjob_1', 'Mjob_2', 'Mjob_3', 'Mjob_4', 'Mjob_5', 'Fjob_1',
       'Fjob_2', 'Fjob_3', 'Fjob_4', 'Fjob_5', 'reason_1', 'reason_2',
       'reason_3', 'reason_4', 'guardian_1', 'guardian_2', 'guardian_3',
       'traveltime', 'studytime', 'failures', 'schoolsup_1', 'schoolsup_2',
       'famsup_1', 'famsup_2', 'paid_1', 'paid_2', 'activities_1',
       'activities_2', 'nursery_1', 'nursery_2', 'higher_1', 'higher_2',
       'internet_1', 'internet_2', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major_1', 'major_2'],
      dtype='object')

In [68]:
def parse(data, encoder):
    """
    Parsing data to the same format as in "students-all.csv"
    
    returns: nicks array and dataframe
    """
    nicks = np.array(data['Nick'])
    mails = np.array(data['Feedback mail'])
    data = data.iloc[:, 2:-1]

    column_names = [
        'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob',
        'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
        'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'internet',
        'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences',
        'G1', 'G2', 'G3'
    ]

    data.columns = column_names

    # deafults
    data['school'] = np.repeat("GP", data.shape[0])
    data['major'] = np.repeat("mat", data.shape[0])
    data['higher'] = np.repeat("yes", data.shape[0])

    # corect order
    data = data[[
        'school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
        'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
        'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc',
        'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major'
    ]]
    
    data['age'] = pd.to_numeric(data['age'])
    data['sex'] = np.where(data.sex == 'Male', 'M', 'F')
    data['address'] = np.where(data.address == 'Rural', 'R', 'U')
    data['famsize'] = np.where(data.famsize == 'more than 3', 'GT3', 'LE3')
    data['Pstatus'] = np.where(data.Pstatus == 'living apart', 'A', 'T')

    d1 = {
        'none': 0,
        'primary education': 1,
        'middle school': 2,
        'high school': 3,
        'higher education': 4
    }

    data['Medu'] = [d1[item] for item in list(data.Medu)]
    data['Fedu'] = [d1[item] for item in list(data.Fedu)]

    d2 = {
        'teacher': 'teacher',
        'healthcare': 'health',
        'civil services': 'civil',
        'home': 'at_home',
        'other': 'other'
    }

    data['Mjob'] = [d2[item] for item in list(data.Mjob)]
    data['Fjob'] = [d2[item] for item in list(data.Fjob)]

    d3 = {
        'close to home': 'home',
        'school reputation': 'reputation',
        'course preference': 'course',
        'other': 'other'
    }

    data['reason'] = [d3[item] for item in list(data.reason)]
    
    d4 = {'Mother':'mother', 'Father':'father', 'other':'other'}
    
    data['guardian'] = [d4[item] for item in list(data.guardian)]

    d5 = {'< 15':1, '15-30':2, '30-60':3,'60 >':4}
    
    data['traveltime'] = [d5[item] for item in list(data.traveltime)]
    
    d6 = {'< 2':1, '2-5':2, '5-10':3,'10 >':4}
    
    data['studytime'] = [d6[item] for item in list(data.studytime)]
    
    data['G1'] = np.array((pd.to_numeric(data.G1)/5)*20 , dtype = 'int64')
    data['G2'] = np.array((pd.to_numeric(data.G2)/5)*20 , dtype = 'int64') 
    data['G3'] = np.array((pd.to_numeric(data.G3)/5)*20 , dtype = 'int64')
    
    data['absences'] = np.array(pd.to_numeric(data.absences) , dtype = 'int64')
    data['failures'] = np.array(data.failures, dtype = 'int64')
    data['famrel']   = np.array(data.famrel, dtype = 'int64')
    data['freetime'] = np.array(data.freetime, dtype = 'int64')
    data['goout']    = np.array(data.goout, dtype = 'int64')
    data['Dalc']     = np.array(data.Dalc, dtype = 'int64')
    data['Walc']     = np.array(data.Walc, dtype = 'int64')
    data['health']   = np.array(data.health, dtype = 'int64')
    
    
    data_transformed = encoder.transform(data)
    
    return nicks, mails, data_transformed

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
creds = flow.run_local_server(port=0)




Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=31984095153-kph5j29muu11g829pfi3jnncqknk5qeo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A43543%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=ilrTWZycfkiiQVpPQIrKrz1rgfPOZh&access_type=offline


In [8]:
service = build('sheets', 'v4',credentials=creds)
sheet = service.spreadsheets()

In [9]:
SAMPLE_SPREADSHEET_ID = '1e1tWLI0vD05bUj-wLWicOnl0iU-GWz0aaWEtRDlTQ2M'

cols = "AG"
rows = 4000

SAMPLE_RANGE_NAME = 'A1:'+ cols + str(rows)

In [10]:
SAMPLE_RANGE_NAME

'A1:AG4000'

In [74]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])
data = pd.DataFrame(values)
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data.head()

,Sygnatura czasowa,Nick,Sex,Age,Adress,Family size,Parent status,Mother education,Father education,Mother job,Father job,Why do you study on MiNI?,Who is your legal guardian?,How long is your comute (in minutes),Study time (in hours),Failed courses,Do you take extra lessons?,Do your family supports your education?,"Do you take courses that you pay for? (dancing, PE, etc...)","Do you have extra activities? (organisations, sport, etc... )",Did you attend preschool?,Do you have internet access at home?,Quality of family relationships,Free time after school,Going out with friends,Workday alcohol consumption,Weekend alcohol consumption,Health,Number of school absences (0-90),3rd semester grade (2-5),4rd semester grade (2-5),Grade up to this date (2-5),Feedback mail
1,2020-04-18 22:36:19,testuś,Male,19,Rural,Less or equal than 3,living apart,middle school,middle school,civil services,other,school reputation,other,15-30,10 >,1,No,No,Yes,Yes,No,No,2,3,2,4,2,5,13,3.14,4.19,3.90,AlcoholRomantics@gmail.com
2,2020-04-18 22:40:00,testuś2,Female,24,Urban,more than 3,living toogether,none,middle school,other,home,other,Mother,60 >,5-10,1,Yes,Yes,No,No,No,Yes,3,3,3,3,3,3,4,4,4,4,AlcoholRomantics@gmail.com
3,2020-04-19 10:49:35,test3,Male,21,Urban,more than 3,living toogether,primary education,none,healthcare,teacher,course preference,Father,15-30,2-5,2,No,Yes,Yes,No,No,No,2,3,1,5,5,1,53,4,5,3,AlcoholRomantics@gmail.com
4,2020-04-19 12:09:57,test4,Male,20,Urban,more than 3,living toogether,middle school,primary education,teacher,teacher,close to home,Mother,30-60,5-10,3,No,No,Yes,Yes,No,Yes,4,3,3,3,3,3,32,4,3,2,AlcoholRomantics@gmail.com
5,2020-04-19 12:17:25,test_czy_submit_dziala,Male,20,Urban,more than 3,living apart,none,higher education,healthcare,healthcare,close to home,other,15-30,5-10,3,No,Yes,Yes,Yes,Yes,Yes,4,3,3,3,3,3,34,2,3,4,AlcoholRomantics@gmail.com


In [75]:
n, m, df_parsed = parse(data, encoder)

/home/jakwisn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jakwisn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [82]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

target = np.where(y=='yes', 1, 0)

X_train, X_test, y_train, y_test = train_test_split(df_one_hot,
                                                    target, test_size = 0.2, random_state = 666)

xgb_model = xgb.XGBClassifier(max_depth = 7,
                              booster = "dart",
                              colsample_bytree = 0.3,
                              learning_rate = 0.39,
                              reg_alpha = 0.9,
                              reg_lambda = 1.8,
                              subsample = 0.8)
xgb_model.fit(X_train,y_train)

#y_prob = xgb_model.predict_proba(X_test)[:,1]

xgb_model.predict_proba(X_test)[:,1]

array([0.8812787 , 0.22040395, 0.4031762 , 0.6974458 , 0.08547182,
       0.8501033 , 0.2860232 , 0.40355185, 0.963621  , 0.96765625,
       0.47420335, 0.39759827, 0.7111148 , 0.90773994, 0.59242994,
       0.8735725 , 0.07239775, 0.12113819, 0.44955614, 0.48650867,
       0.59061754, 0.08185454, 0.09065181, 0.5949267 , 0.13763414,
       0.08202299, 0.02887847, 0.94923294, 0.43247432, 0.01008812,
       0.29716554, 0.23240782, 0.06220552, 0.02540871, 0.11959811,
       0.7405803 , 0.10589141, 0.2609206 , 0.01105169, 0.05751356,
       0.00235477, 0.33035758, 0.81917185, 0.17772758, 0.06577341,
       0.05620021, 0.62204057, 0.09689726, 0.3022899 , 0.84786874,
       0.02097002, 0.08864792, 0.04216398, 0.41542283, 0.9726328 ,
       0.24189626, 0.2986846 , 0.12508363, 0.02411472, 0.01636192,
       0.57883567, 0.7920542 , 0.02981267, 0.21411234, 0.34726354,
       0.2462636 , 0.7554271 , 0.4945621 , 0.08956128, 0.5684176 ,
       0.40201348, 0.5199904 , 0.08388573, 0.03952473, 0.11570

In [83]:
y_predict_prob

array([0.10947108, 0.07915692, 0.04496526, ..., 0.6974458 , 0.03881077,
       0.04744126], dtype=float32)

In [85]:
xgb_model.predict_proba(df_parsed)[:,1]

array([0.6418716 , 0.4799034 , 0.32430822, 0.24649513, 0.52584857,
       0.49768522, 0.88412386, 0.4238217 ], dtype=float32)